In [6]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to scrape product listing page
def scrape_product_listing(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    products = soup.find_all('div', {'data-component-type': 's-search-result'})

    for product in products:
        # Extract product information
        product_url_elem = product.find('a', {'class': 'a-link-normal a-text-normal'})
        if product_url_elem:
            product_url = 'https://www.amazon.in' + product_url_elem['href']
        else:
            product_url = 'Not available'
        
        product_name_elem = product.find('span', {'class': 'a-size-medium a-color-base a-text-normal'})
        product_name = product_name_elem.text if product_name_elem else 'Not available'
        
        product_price_elem = product.find('span', {'class': 'a-price-whole'})
        product_price = product_price_elem.text if product_price_elem else 'Not available'

        rating_elem = product.find('span', {'class': 'a-icon-alt'})
        rating = rating_elem.text.split()[0] if rating_elem else 'Not available'

        reviews_elem = product.find('span', {'class': 'a-size-base'})
        reviews = reviews_elem.text if reviews_elem else '0'

        # Create a dictionary to store the product information
        product_data = {
            'Product URL': product_url,
            'Product Name': product_name,
            'Product Price': product_price,
            'Rating': rating,
            'Number of Reviews': reviews
        }

        # Append the product data to the CSV file
        with open('product_data.csv', 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=product_data.keys())
            writer.writerow(product_data)

# Main scraping logic
base_url = 'https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_'
num_pages = 20
num_products_per_page = 10

# Create the CSV file and write the header row
with open('product_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Product URL', 'Product Name', 'Product Price', 'Rating', 'Number of Reviews'])

for page in range(1, num_pages + 1):
    url = base_url + str(page)
    print('Scraping page', page)
    scrape_product_listing(url)

    if (page * num_products_per_page) > 200:
        break

# URLs received in the above case
product_urls = [
    # List of product URLs
]

# Function to scrape individual product page
def scrape_product_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract product information
    product_description = soup.find('div', {'id': 'productDescription'}).text.strip()
    asin = soup.find('th', text='ASIN').find_next_sibling('td').text.strip()
    manufacturer = soup.find('th', text='Manufacturer').find_next_sibling('td').text.strip()

    # Print additional product information
    print('Product Description:', product_description)
    print('ASIN:', asin)
    print('Manufacturer:', manufacturer)
    print('---')

for url in product_urls:
    print('Scraping product page:', url)
    scrape_product_page(url)


Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20


In [82]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

base_url = 'https://www.amazon.in/s'
params = {
    'k': 'bags',
    'crid': '2M096C61O4MLT',
    'qid': '1653308124',
    'sprefix': 'ba,aps,283',
    'ref': 'sr_pg_'
}

product_count = 0
total_pages = 200

product_links = list()
product_names = list()
product_prices = list()
product_reviews = list()
product_ratings = list()
for page in tqdm(range(1, total_pages + 1)):
    params['ref'] = 'sr_pg_' + str(page)
    response = requests.get(base_url, params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    product_list = soup.find_all('div', {'data-component-type': 's-search-result'})

    for product in product_list:
        product_url = 'https://www.amazon.in' + product.find('a', {'class': 'a-link-normal'})['href']
        product_name = product.find('span', {'class': 'a-size-medium'}).text.strip()
        product_price = product.find('span', {'class': 'a-price-whole'}).text.replace(',', '')
        rating = product.find('span', {'class': 'a-icon-alt'}).text.split()[0]
        num_reviews = product.find('span', {'class': 'a-size-base'}).text.replace(',', '')
        product_links.append(product_url)
        product_names.append(str(product_name))
        product_prices.append(product_price)
        product_reviews.append(num_reviews)
        product_ratings.append(rating)
        
        product_count += 1
        if product_count >= 250:
            break

        
        # function calls to display all necessary product information


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [03:00<00:00,  1.11it/s]


In [83]:
len(product_links)

276

In [84]:
assert (len(product_links) == len(product_names) == len(product_prices) == len(product_reviews) == len(product_ratings))

In [89]:
my_dict = {
    "product_url":product_links,
    "product_name":product_names,
    "product_price":product_prices,
    "product_rating":product_ratings,
    "product_reviews":product_reviews,
}

In [90]:
pd.DataFrame(my_dict).to_csv("product_info.csv",index = False)